---
title: TRABAJO PRÁCTICO N° 2
subtitle: Histogramas, Kernels & Métodos no supervisados usando la EPH
author: Ricardo Javier Gutiérrez Vistín - Martín Gabriel Cargnel
date-format: "MMM, YYYY"
lang: es
execute:
  echo: true
  include: false
  fig-align: center
format:
  pdf:
    documentclass: scrartcl
    toc-depth: 2
    number-sections: true
---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns

Link al repositorio: https://github.com/mcargnel/Big-Data-UBA-Grupo-001/tree/main/TP1

# Creación de variables, histogramas, kernels y resumen de la base de datos final

In [25]:
#df_usu_hogar = pd.read_excel('data/usu_hogar_T125.xlsx')
df_individual_t105 = pd.read_stata('data/Individual_t105.dta')
df_individual_t125 = pd.read_excel('data/usu_individual_T125.xlsx')

In [ ]:
df_individual_t125.columns = df_individual_t125.columns.str.lower()
df_individual_t105.columns = df_individual_t105.columns.str.lower()
df_individual_t105_filt = df_individual_t105[df_individual_t105['region'] == 'Gran Buenos Aires']

df_individual_t125_filt = df_individual_t125[df_individual_t125['region'] == 1]

df_total = pd.concat([df_individual_t105_filt, df_individual_t125_filt])

df_total.shape

(16665, 243)

In [27]:
variables_identificatorias = ['CODUSU', 'nro_hogar', 'componente']

variables_dadas = ['CH04', 'CH06', 'CH07', 'CH08', 'NIVEL_ED', 'ESTADO', 'CAT_INAC', 'IPCF']

variables_elegidas = ['PP04A', 'PP05B2_ANO', 'pp07c','pp07g_59','ITF','pp07j','PP11O']

total_variables = variables_identificatorias + variables_dadas + variables_elegidas
total_variables = [i.lower() for i in total_variables]
len(total_variables)

18

In [28]:
df_total = df_total[['ano4']+total_variables]

In [29]:
df_total['ch04'] = df_total['ch04'].replace({1: 'Varón', 2: 'Mujer'})

In [41]:
df_total['ch06'] = df_total['ch06'].replace({'Menos de 1 año': 0})
df_total['ch06'] = df_total['ch06'].astype(int) 
df_total = df_total.loc[df_total['ch06'] >= 0]

In [43]:
df_total['ch07'] = df_total['ch07'].replace({1: 'Unido', 2: 'Casado', 3: 'Separado o divorciado',
                                             4: 'Viudo', 5:'Soltero'})

In [44]:
df_total['ch08'] = df_total['ch08'].replace({1.0: 'Obra social (incluye PAMI)',
                                             2.0: 'Mutual/Prepaga/Servicio de emergencia',
                                             3.0: 'Planes y seguros públicos',
                                             4.0: 'No paga ni le descuentan',
                                             9.0: 'Ns./Nr.',
                                             12.0: 'Obra social y mutual/prepaga/servicio de emergencia',
                                             123.0: 'Obra social y planes y seguros públicos'
                                             })

In [45]:
df_total['nivel_ed'] = df_total['nivel_ed'].replace({1: 'Primaria Incompleta (incluye educación especial)',
                                                    2: 'Primaria Completa',
                                                    3: 'Secundaria Incompleta',
                                                    4: 'Secundaria Completa',
                                                    5:'Superior Universitaria Incompleta',
                                                    6:'Superior Universitaria Completa',
                                                    7:'Sin instrucción'})

In [46]:
df_total['estado'] = df_total['estado'].replace({1: 'Ocupado',
                                                    2: 'Desocupado',
                                                    3: 'Inactivo',
                                                    4: 'Menor de 10 años',
                                                    0:'Entrevista individual no realizada (no respuesta al cuestion'
                                                    })

In [47]:
df_total['cat_inac'] = df_total['cat_inac'].replace({1: 'Jubilado/pensionado',
                                                    2: 'Rentista',
                                                    3: 'Estudiante',
                                                    4: 'Ama de casa',
                                                    5: 'Menor de 6 años',
                                                    6: 'Discapacitado',
                                                    7: 'Otros'
                                                    })

In [52]:
df_total['pp04a'] = df_total['pp04a'].replace({1.0: 'Estatal',
                                               '...estatal?': 'Estatal',
                                                    2.0: 'Privada?',
                                                    '...privada?':'Privada',
                                                    3.0: 'Otro',
                                                    '...de otro tipo? (especificar)': 'Otro'
                                                    })

In [53]:
df_total['pp07c'] = df_total['pp07c'].replace({1.0: 'Sí (incluye changa, trabajo transitorio, por tarea u obra, s',
                                               2.0: 'No (Incluye permanente, fijo, estable, de planta)',
                                               9.0: 'Ns./Nr.'
                                                })

In [54]:
df_total['pp07j'] = df_total['pp07j'].replace({1.0: 'Dia',
                                               2.0: 'Noche',
                                               3.0:'Otros',
                                               '...de día? (mañana/tarde)': 'Dia',
                                               '...de noche':'Noche',
                                               '...de otro tipo? (rotativo, día y noche, guardias con franco':'Otros'
                                                                                                        })

In [55]:
df_total['pp11o'] = df_total['pp11o'].replace({1.0: 'Despido/cierre',
                                               2.0: 'Por retiro voluntario del sector público',
                                               3.0:'Jubilación',
                                               4.0:'Fin del trabajo temporario/estacional',
                                               5.0:'Le pagaban poco/no le pagaban',
                                               6.0:'Malas relaciones laborales/malas condiciones de trabajo (ins',
                                               7:'Renuncia obligada/pactada',
                                               8.0: 'Otro',
                                               9:'Por razones personales'
                                                })

In [113]:
df_tabla_adulto = pd.read_excel('/Users/mcargnel/Documents/mea/Big-Data-UBA-Grupo-001/TP1/data/tabla_adulto_equiv.xlsx', skiprows=3)
df_tabla_adulto.dropna(inplace=True)
df_tabla_adulto['Edad'] = df_tabla_adulto['Edad'].str.replace(' años', '').replace('año', '')

In [114]:
rangos = [col for col in df_tabla_adulto['Edad'] if ' a ' in col]
tabla_rangos = df_tabla_adulto[df_tabla_adulto['Edad'].isin(rangos)]

rangos_expandidos = []

for _, row in tabla_rangos.iterrows():
    start, end = map(int, row["Edad"].split(" a "))
    for age in range(start, end + 1):
        rangos_expandidos.append({
            "Edad": age,
            "Mujeres": row["Mujeres"],
            "Varones": row["Varones"]
        })

last_row = tabla_rangos.iloc[-1]
for age in range(76, 101):
    rangos_expandidos.append({
        "Edad": age,
        "Mujeres": last_row["Mujeres"],
        "Varones": last_row["Varones"]
    })

rangos_expandidos_df = pd.DataFrame(rangos_expandidos)

In [115]:
df_tabla_adulto = pd.concat([df_tabla_adulto,rangos_expandidos_df])

In [118]:
df_tabla_adulto = df_tabla_adulto[~df_tabla_adulto['Edad'].isin(rangos +['más de 75'])]
df_tabla_adulto['Edad'] = df_tabla_adulto['Edad'].replace({'1año':1, 'Menor de 1 año': 0})
df_tabla_adulto['Edad'] = df_tabla_adulto['Edad'].astype(int)

/var/folders/c6/f1ghz9qx1bbb4_2pszbddg800000gn/T/ipykernel_11434/4118338592.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tabla_adulto['Edad'] = df_tabla_adulto['Edad'].replace({'1año':1, 'Menor de 1 año': 0})
/var/folders/c6/f1ghz9qx1bbb4_2pszbddg800000gn/T/ipykernel_11434/4118338592.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tabla_adulto['Edad'] = df_tabla_adulto['Edad'].astype(int)


In [119]:
df_tabla_adulto = df_tabla_adulto.melt(
    id_vars=['Edad'],
    value_vars=['Mujeres', 'Varones'],
    value_name='adulto_equiv',
    var_name='ch04'
)

In [120]:
df_tabla_adulto['ch04'] = df_tabla_adulto['ch04'].replace({'Mujeres': 'Mujer', 'Varones':'Varón'})
df_tabla_adulto.columns = ['ch06', 'ch04', 'adulto_equiv']

In [121]:
df_total = df_total.merge(df_tabla_adulto, on=['ch06', 'ch04'], how='left')

In [93]:
df_total["ad_equiv_hogar"] = (
    df_total.groupby(["codusu", "nro_hogar"])["adulto_equiv"]
      .transform("sum")
)

In [ ]:
# respondieron itf

respondieron = df_total[df_total['itf'] != 0]

# no respondieron itf
norespondieron = df_total[df_total['itf'] == 0]
norespondieron['ano4'].value_counts()

ano4
2025.0    2857
2005.0     113
Name: count, dtype: int64

In [145]:
respondieron['ingreso_necesario'] = np.where(respondieron['ano4'] == 2005.0, respondieron['ad_equiv_hogar'] * 205.07,respondieron['ad_equiv_hogar'] * 365177)  

/var/folders/c6/f1ghz9qx1bbb4_2pszbddg800000gn/T/ipykernel_11434/2959864343.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  respondieron['ingreso_necesario'] = np.where(respondieron['ano4'] == 2005.0, respondieron['ad_equiv_hogar'] * 205.07,respondieron['ad_equiv_hogar'] * 365177)


In [146]:
respondieron['pobre'] = np.where(respondieron['itf'] <respondieron['ingreso_necesario'] , 1, 0)

/var/folders/c6/f1ghz9qx1bbb4_2pszbddg800000gn/T/ipykernel_11434/1349639960.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  respondieron['pobre'] = np.where(respondieron['itf'] <respondieron['ingreso_necesario'] , 1, 0)


In [148]:
#| label: tab-pobres
#| tbl-cap: Tabla con la cantidad y proporción de pobres por año
#| echo: false
#| include: true

pobres_df = respondieron[['ano4','pobre']].value_counts().reset_index().sort_values('ano4')
pobres_df['prop'] = round(pobres_df['count'] /pobres_df.groupby('ano4')['count'].transform('sum'), 4)
pobres_df.columns = ['Año', 'Pobre', 'Cantidad', 'Proporción']
pobres_df

,Año,Pobre,Cantidad,Proporción
0,2005.0,0,6878,0.7340
2,2005.0,1,2493,0.2660
1,2025.0,0,2950,0.6889
3,2025.0,1,1332,0.3111


# Métodos No Supervisados